Finetune with MBart 


Tryout from preprocessing custom dataset for huggingface!!!!

In [1]:
from datasets import load_dataset, concatenate_datasets
import transformers
from transformers import MBartModel, MBartTokenizer, MBartConfig, pipeline, Trainer, TrainingArguments, MBartForConditionalGeneration, DataCollatorForSeq2Seq
import evaluate


Dataload using Huggingface 

In [2]:
train_data = load_dataset("json", data_files ="/home/sumire/main/discourse_context_mt/data/BSD-master/train.json")
dev_data = load_dataset("json", data_files ="/home/sumire/main/discourse_context_mt/data/BSD-master/dev.json")
test_data = load_dataset("json", data_files ="/home/sumire/main/discourse_context_mt/data/BSD-master/test.json")

Using custom data configuration default-0b58fbb4fdc24ac7
Found cached dataset json (/home/sumire/.cache/huggingface/datasets/json/default-0b58fbb4fdc24ac7/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-6b02b9f67a1bac72
Found cached dataset json (/home/sumire/.cache/huggingface/datasets/json/default-6b02b9f67a1bac72/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-0b1429c816b88702
Found cached dataset json (/home/sumire/.cache/huggingface/datasets/json/default-0b1429c816b88702/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
train_data['dev']=dev_data['train']
train_data['test']=test_data['train']
data=train_data
data

DatasetDict({
    train: Dataset({
        features: ['id', 'tag', 'title', 'original_language', 'conversation'],
        num_rows: 670
    })
    dev: Dataset({
        features: ['id', 'tag', 'title', 'original_language', 'conversation'],
        num_rows: 69
    })
    test: Dataset({
        features: ['id', 'tag', 'title', 'original_language', 'conversation'],
        num_rows: 69
    })
})

In [4]:
inputs = [sent['en_sentence'] for doc in data["train"]["conversation"] for sent in doc]
targets = [sent['ja_sentence'] for doc in data["train"]["conversation"] for sent in doc]

Preprocess Using Huggingface

In [16]:
model_checkpoint = "facebook/mbart-large-cc25"
configuration = MBartConfig()
tokenizer = MBartTokenizer.from_pretrained(model_checkpoint, src_lang="en_XX", tgt_lang="ja_XX")
model = MBartForConditionalGeneration(configuration).from_pretrained(model_checkpoint)

max_length = 128

def preprocess_function(data): # data should be splitted into train / dev / test internally
    inputs = [sent['en_sentence'] for doc in data["conversation"] for sent in doc]
    targets = [sent['ja_sentence'] for doc in data["conversation"] for sent in doc]
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=max_length, truncation=True
    )
    return model_inputs

loading file sentencepiece.bpe.model from cache at /home/sumire/.cache/huggingface/hub/models--facebook--mbart-large-cc25/snapshots/2df0e6dd8a0e7f6df056fe4d0d95941a04b64e4f/sentencepiece.bpe.model
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at /home/sumire/.cache/huggingface/hub/models--facebook--mbart-large-cc25/snapshots/2df0e6dd8a0e7f6df056fe4d0d95941a04b64e4f/config.json
Model config MBartConfig {
  "_name_or_path": "facebook/mbart-large-cc25",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": true,
  "architectures": [
    "MBartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 

In [17]:
tokenized_datasets = data.map(
    preprocess_function,
    batched=True,
    remove_columns=data["train"].column_names,
)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [18]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 20000
    })
    dev: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2051
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2120
    })
})

DataCollator

In [19]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt") # not tf

Define Batch using Datacollator
###To Chryssa, decoder_input_ids now exists!####

In [20]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(1, 3)])
batch.keys()

# 'decoder_input_ids' is missing

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

In [21]:
batch["labels"]

tensor([[   572,   6808,    154,   1395,   2128,    610,  34606,   5182,     30,
              2, 250012],
        [     6,  38566, 160517,  14362,  55826,     30,      2, 250012,   -100,
           -100,   -100]])

In [22]:
batch["decoder_input_ids"]

tensor([[250012,    572,   6808,    154,   1395,   2128,    610,  34606,   5182,
             30,      2],
        [250012,      6,  38566, 160517,  14362,  55826,     30,      2, 250012,
              1,      1]])

In [23]:
for i in range(1, 3):
    print(tokenized_datasets["train"][i]["labels"])

[572, 6808, 154, 1395, 2128, 610, 34606, 5182, 30, 2, 250012]
[6, 38566, 160517, 14362, 55826, 30, 2, 250012]


Fine-tune using Trainer

In [24]:
training_args = TrainingArguments(
    output_dir='./results',          
    logging_dir='./logs',            
    num_train_epochs=3,              
    per_device_train_batch_size=16,  
    per_device_eval_batch_size=64,   
    warmup_steps=500,                
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,                         
    args=training_args,                  
    train_dataset=tokenized_datasets["train"],        
    eval_dataset=tokenized_datasets["dev"],            
    data_collator=data_collator,
    tokenizer=tokenizer
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


RuntimeError: CUDA out of memory. Tried to allocate 978.00 MiB (GPU 0; 11.91 GiB total capacity; 11.01 GiB already allocated; 310.94 MiB free; 11.03 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [25]:
trainer.train()

***** Running training *****
  Num examples = 20000
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 939
  Number of trainable parameters = 610851840


RuntimeError: Caught RuntimeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/home/sumire/miniconda3/envs/mt2022_server/lib/python3.8/site-packages/torch/nn/parallel/parallel_apply.py", line 61, in _worker
    output = module(*input, **kwargs)
  File "/home/sumire/miniconda3/envs/mt2022_server/lib/python3.8/site-packages/torch/nn/modules/module.py", line 1130, in _call_impl
    return forward_call(*input, **kwargs)
  File "/home/sumire/miniconda3/envs/mt2022_server/lib/python3.8/site-packages/transformers/models/mbart/modeling_mbart.py", line 1353, in forward
    outputs = self.model(
  File "/home/sumire/miniconda3/envs/mt2022_server/lib/python3.8/site-packages/torch/nn/modules/module.py", line 1130, in _call_impl
    return forward_call(*input, **kwargs)
  File "/home/sumire/miniconda3/envs/mt2022_server/lib/python3.8/site-packages/transformers/models/mbart/modeling_mbart.py", line 1218, in forward
    encoder_outputs = self.encoder(
  File "/home/s

In [29]:
# Training without using Trainer
##Doesn't work###

inputs = [sent['en_sentence'] for doc in data["train"]["conversation"] for sent in doc]
targets = [sent['ja_sentence'] for doc in data["train"]["conversation"] for sent in doc]

inputs = tokenizer(inputs, text_target=targets, return_tensors="pt", padding=True)
print (inputs)

model(**inputs)

{'input_ids': tensor([[  2673,    903,     83,  ...,      1,      1,      1],
        [  2646,   9351,     83,  ...,      1,      1,      1],
        [ 25689,    398,    237,  ...,      1,      1,      1],
        ...,
        [    87,     25,   1181,  ...,      1,      1,      1],
        [240095,    764,   1221,  ...,      1,      1,      1],
        [    87,  15673,    221,  ...,      1,      1,      1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([[     6, 135855,     37,  ...,      1,      1,      1],
        [   572,   6808,    154,  ...,      1,      1,      1],
        [     6,  38566, 160517,  ...,      1,      1,      1],
        ...,
        [     6,  81766, 102535,  ...,      1,      1,      1],
        [     6,  47181,    281,  ...,      1,      1,      1],
       

: 

: 

BLEU metrics

In [ ]:
import evaluate 


In [ ]:
metric = evaluate.load("sacrebleu")